In [197]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from pprint import pprint
%pylab inline
import pandas as pd
import os
from collections import Counter
from itertools import chain
import json
import seaborn as sns
import matplotlib.pyplot as plt
import ast

import gc
print(os.getcwd())

Populating the interactive namespace from numpy and matplotlib
/Users/ninastijepovic/Desktop/assignment1


In [198]:
def get_labels(fname):
    label_mappings = {}
    with open(fname,'r+') as f:
        
        for line in f.readlines():
            
            line = line.rstrip().lower()
            
            splitted = line.split('|')
            splitted = [x.strip().rstrip().lower() for x in splitted]
            if len(splitted) == 2:
                if splitted[1] =='label':
                    #print('yeah')
                    continue
                label = int(splitted[0])
                name = splitted[1].rstrip().lower()
                label_mappings[name] = label
                
            else:
                break
    print (label_mappings)
    return label_mappings
                


def get_sensor_values(fname,device='./datasets/tina/log2'):
    '''Get sensor values function. Parses the .txt log and puts all sensors in a csv
        args:
            fname the filename of the sensor log
        
        output:
            sensor_dict a dictionary where the keys are the sensors and the values are label-vector-timestamp
    '''
    sensor_dict = {}
    device = fname.split('/')[len(fname.split('/'))-1]
    print('The device is %s'%device)
    labels_dict = {}
    label_names = get_labels(fname)
    print(label_names)
    with open(fname,'r+') as f:
        for line in f.readlines():
            splitted = line.split('|')
            if len(splitted) != 4:
                #in case we are before our data to avoid parsing errors
                #print ("split different than 4")
                continue
            splitted = [x.rstrip().lower() for x in splitted]
            label,sensorname,vector,timestamp = splitted
            if label not in labels_dict.keys():
                labels_dict[label] = []
            else:
                if label == 'statusid':
                    continue
                tmp_list = labels_dict.get(label)
#                 #print (label)
#                 print (tmp_list)
                tmp_list.append(['interval_label',
                                     device,
                                     label_names.keys()[label_names.values().index(int(label))],
                                     timestamp
                                    ])
#                 except:
#                     print ("fucked up label")
            ### same as above this is actually the header  list.
            if sensorname == 'sensorname':
#                 print ("fuck")
                continue
                
#             print ("not fuck")
            vector = json.loads(vector)
            if sensorname not in sensor_dict.keys():
                sensor_dict[sensorname] = []
            else:
                tmp_list = sensor_dict.get(sensorname)
                tmp_list.append([label,vector,timestamp])
                sensor_dict[sensorname] = tmp_list
            
    return sensor_dict,labels_dict

def generate_labels(values,header=['device','label','label_start','label_end'],fname='./datasets/tina/csv/labels122.csv'):
    label_mapping = get_sensor_values(dataset_path)
    
    with open(fname,'w') as f:
        f.write(''.join([x+',' for x in header])+'\n')
        for key in values.keys():
            label_values = values[key]
            if len(label_values) == 0:
                continue
            label_start = label_values[0][3]
            label_end = label_values[-1][3]
            for v in label_values:
                if key == '1':
                    key_map = 'Sit'
                elif key == '3':
                    key_map = 'Walk'
                elif key == '4':
                    key_map = 'Cycling'
                else:
                    print(key,type(key))
                f.write('%s,%s,%s,%s\n'%(v[1],key_map,label_start,label_end))

def generate_csv(values,header=['x','y','z','timestamp'],label_only='1',fname='./datasets/tina/csv/gyroscope.csv'):
    '''Creates a .csv at a specified path for a certain input'''
    '''
        args:
            values    the sensor data values , a list of lists in our case
            header    the header of the .csv . Default is label,x,y,z,timestamp 
            fname     the path and filename to save the .csv file
        
            
    '''
    with open(fname,'w') as f:
        f.write(''.join([x+',' for x in header])+'\n')
        for v in values:
            
#             if str(v[0]) != str(label_only):
#                 continue
            v[2] = float(v[2])
            x = str(v[1]).replace('[','').replace(']','').replace(' ','')
            if header[0] =='light_intensity':
                x = x.replace(',0.0','')    
            line = '%s,%d\n'%(x,v[2])
            f.write(line)


In [199]:
#Directory tree should be :
# |-WORKSPACE
# |--thisnotebok.ipynb
# |--datasets
# |--toy_data
# |---sensorLog_01.txt
# |---csv
# |----sensorname.csv
# |----anothersensorname.csv
dataset_folder = './datasets'
current_dataset = 'tina'
dataset_fname = 'log2'

dataset_path = os.path.join(dataset_folder+'/'+current_dataset,dataset_fname)
csv_path = os.path.join(dataset_folder+'/'+current_dataset,'csv/')
if not os.path.exists(csv_path):
    os.mkdir(csv_path)
cnt = 0
print(csv_path)

./datasets/tina/csv/


In [200]:
sensor_dict,labels_dict = get_sensor_values(dataset_path)

The device is log2
{'walk': 3, 'cycling': 4, 'sit': 1}
{'walk': 3, 'cycling': 4, 'sit': 1}


In [201]:
sensor_names = sensor_dict.keys()
print('==============================================================')
print('Sensors are:',json.dumps(sensor_names,indent=2))

Sensors are: [
  "ak09911 magnetometer", 
  "orientation", 
  "gravity -wakeup secondary", 
  "bmi160 gyroscope", 
  "rmd", 
  "bmi160 gyroscope uncalibrated -wakeup secondary", 
  "linear acceleration -wakeup secondary", 
  "linear acceleration", 
  "bmi160 accelerometer uncalibrated -wakeup", 
  "geomagnetic rotation vector -wakeup secondary", 
  "gravity", 
  "pickup", 
  "ak09911 magnetometer uncalibrated -wakeup secondary", 
  "rotation vector", 
  "orientation -wakeup secondary", 
  "tcs3400 als -wakeup secondary", 
  "basic gestures", 
  "bmi160 gyroscope uncalibrated", 
  "tcs3400 rgb_ct", 
  "game rotation vector -wakeup secondary", 
  "bmi160 accelerometer", 
  "rotation vector -wakeup secondary", 
  "tcs3400 als", 
  "bmi160 gyroscope -wakeup secondary", 
  "ak09911 magnetometer uncalibrated", 
  "motion accel", 
  "bmi160 accelerometer uncalibrated -wakeup secondary", 
  "game rotation vector", 
  "geomagnetic rotation vector", 
  "gps", 
  "bmi160 accelerometer -wakeup sec

In [202]:
generate_labels(labels_dict,fname=csv_path+'labels122.csv')


The device is log2
{'walk': 3, 'cycling': 4, 'sit': 1}
{'walk': 3, 'cycling': 4, 'sit': 1}


In [203]:
for name in sensor_names:
    fname = name.replace(' ','_')+'.csv'
    if 'light' in name:
        generate_csv(sensor_dict[name],header=['light_intensity','timestamp'],fname=csv_path+fname)
    elif 'rotation' in name:
        generate_csv(sensor_dict[name],header=['x','y','z','theta','phi','timestamp'],fname = csv_path +fname)
    elif 'step' in name:
        generate_csv(sensor_dict[name],header=['value','timestamp'],fname=csv_path+fname)
    else:    
        generate_csv(sensor_dict[name],fname=csv_path+fname)
    

In [204]:
l=pd.read_csv("./datasets/tina/csv/labels122.csv")

In [205]:
pd.unique(l["label"])

array(['Cycling'], dtype=object)

In [162]:
print (v)

NameError: name 'v' is not defined